In [1]:
import pandas as pd

import sqlite3
conn = sqlite3.connect('./ipl_database.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Iterate over the table names
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    sql_query = 'SELECT * FROM '+ table_name+';'
    df = pd.read_sql_query(sql_query, conn)
    xlsx_file_path = "C:/Users/anubr/Python Projects/IPL/assets_xls/"

    # Convert the DataFrame to a excel file
    df.to_excel(xlsx_file_path+table_name+".xlsx", index=False)

    # Print a message indicating the conversion is complete
    print(f"DataFrame has been successfully converted to CSV file: {xlsx_file_path+table_name}.xls")

    
conn.close()

Table: ipl_ball_by_ball
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_ball_by_ball.xls
Table: ipl_venue_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_list.xls
Table: ipl_venue_innings_1
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_1.xls
Table: ipl_venue_innings_2
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_2.xls
Table: ipl_match_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_match_list.xls
Table: ipl_player_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_player_list.xls


In [11]:
import sqlite3
import data_manipulation
def get_batting_positions_id_innings(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # SQL query to fetch all unique batters in the specified match ID and innings
    query = '''
    SELECT DISTINCT batter
    FROM ipl_ball_by_ball
    WHERE ID = ? AND innings = ?
    ORDER BY overs, ballnumber
    '''

    cursor.execute(query, (match_id, innings))
    batting_positions = [row[0] for row in cursor.fetchall()]

    conn.close()
    return batting_positions

def get_all_players(match_id):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve the team players from the ipl_match_list table for the given match ID
    cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
    team_players = cursor.fetchone()

    conn.close()

    if team_players:
        # Split the string to get the list of players for both teams
        team1_players = team_players[0][1:-1].split(", ")
        team2_players = team_players[1][1:-1].split(", ")
        all_players = team1_players + team2_players
        return all_players
    else:
        print("No match found for the given ID.")
        return None

def get_remaining_players(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve toss information to determine which team is batting first
    cursor.execute("SELECT TossWinner, TossDecision FROM ipl_match_list WHERE ID = ?;", (match_id,))
    toss_data = cursor.fetchone()
    team1, team2 = None, None
    if toss_data:
        toss_winner, toss_decision = toss_data

        # Determine the team batting first based on toss decision
        if toss_decision.lower() == 'bat':
            batting_team = toss_winner
        else:
            # If the decision is to field, the other team is batting first
            cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
            teams_data = cursor.fetchone()
            team1, team2 = teams_data
            batting_team = team1 if toss_winner == team2 else team2

        # Retrieve the players' names from the ipl_match_list table for the batting team
        cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
        players_data = cursor.fetchone()

        conn.close()

        if players_data:
            team1_players_str = players_data[0] if batting_team == team1 else players_data[1]
            team2_players_str = players_data[1] if batting_team == team2 else players_data[0]

            # Convert the string format to a list of player names
            batting_team_players = eval(team1_players_str) if batting_team == team1 else eval(team2_players_str)

            # Get the batters who batted
            batters_list = data_manipulation.get_batting_positions_id_innings(match_id, innings)

            # Remove the batters who batted from the list of all players in the batting team
            remaining_players = [player for player in batting_team_players if player not in batters_list]

            return remaining_players
        else:
            print("No match found for the given ID.")
            return None
    else:
        print("No toss information found for the given ID.")
        return None

# Example usage:
match_id = 1312200
innings = 2
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)
print(len(remaining_players))

# Example usage:
match_id = 1312200
innings = 1
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)

Remaining Players: ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy', 'M Prasidh Krishna', 'YS Chahal']
11
Remaining Players: ['M Prasidh Krishna', 'YS Chahal']


In [59]:
import sqlite3
import ast
import data_manipulation

def remaining_players(ID, innings, batting_list):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    cursor.execute("SELECT TossWinner, TossDecision, Team1, Team2, Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (ID,))
    toss_info = cursor.fetchone()
    
    if toss_info:
        TossWinner, TossDecision, Team1, Team2, Team1Players_str, Team2Players_str = toss_info
        
        Team1Players = ast.literal_eval(Team1Players_str)
        Team2Players = ast.literal_eval(Team2Players_str)

        batting_team_players = Team1Players if TossWinner == Team1 else Team2Players

        if innings == 1:
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]
            else:
                remaining_players_list = [player for player in batting_team_players]
        else:  # innings == 2
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players]
            else:
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]

        conn.close()
        return remaining_players_list
    else:
        conn.close()
        return []

ID = 1312199
innings = 1
batters_list = data_manipulation.get_batting_positions_id_innings(ID, innings)
remaining_players(ID, innings, batters_list)


['YBK Jaiswal',
 'JC Buttler',
 'SV Samson',
 'D Padikkal',
 'SO Hetmyer',
 'R Parag',
 'R Ashwin',
 'TA Boult',
 'YS Chahal',
 'M Prasidh Krishna',
 'OC McCoy']

In [1]:
import search_results as sr
search_dict = sr.extract_elements()

In [2]:
search_dict

{'Players': ['NV Ojha',
  'SM Boland',
  'MA Khote',
  'R Dravid',
  'Yash Dayal',
  'Aman Hakim Khan',
  'JP Faulkner',
  'DJ Harris',
  'AJ Finch',
  'S Gopal',
  'L Balaji',
  'IR Jaggi',
  'AT Rayudu',
  'Anirudh Singh',
  'SS Shaikh',
  'Umran Malik',
  'Shahbaz Ahmed',
  'Kartik Tyagi',
  'SB Wagh',
  'DA Miller',
  'YK Pathan',
  'YA Abdulla',
  'Iqbal Abdulla',
  'W Jaffer',
  'PP Ojha',
  'JPR Scantlebury-Searles',
  'KK Cooper',
  'R Shukla',
  'A Mithun',
  'SM Pollock',
  'AD Mascarenhas',
  'Jaskaran Singh',
  'PA Reddy',
  'AB Barath',
  'ND Doshi',
  'KAJ Roach',
  'D Wiese',
  'JJ Roy',
  'Yashpal Singh',
  'BA Stokes',
  'AS Rajpoot',
  'DJ Malan',
  'R Sathish',
  'CRD Fernando',
  'AM Salvi',
  'M Shahrukh Khan',
  'Gagandeep Singh',
  'A Dananjaya',
  'SP Narine',
  'L Ronchi',
  'DJG Sammy',
  'RV Pawar',
  'MN Samuels',
  'Imran Tahir',
  'Shoaib Akhtar',
  'M Kartik',
  'UT Khawaja',
  'Tilak Varma',
  'RP Meredith',
  'Shakib Al Hasan',
  'Sandeep Sharma',
  'DS

In [3]:
players = sorted(search_dict["Players"])
print(players)

['A Ashish Reddy', 'A Badoni', 'A Chandila', 'A Chopra', 'A Choudhary', 'A Dananjaya', 'A Flintoff', 'A Kumble', 'A Manohar', 'A Mishra', 'A Mithun', 'A Mukund', 'A Nehra', 'A Nortje', 'A Singh', 'A Symonds', 'A Uniyal', 'A Zampa', 'AA Bilakhia', 'AA Chavan', 'AA Jhunjhunwala', 'AA Noffke', 'AB Agarkar', 'AB Barath', 'AB Dinda', 'AB McDonald', 'AB de Villiers', 'AC Blizzard', 'AC Gilchrist', 'AC Thomas', 'AC Voges', 'AD Hales', 'AD Mascarenhas', 'AD Mathews', 'AD Nath', 'AD Russell', 'AF Milne', 'AG Murtaza', 'AG Paunikar', 'AJ Finch', 'AJ Turner', 'AJ Tye', 'AK Markram', 'AL Menaria', 'AM Nayar', 'AM Rahane', 'AM Salvi', 'AN Ahmed', 'AN Ghosh', 'AP Dole', 'AP Majumdar', 'AP Tare', 'AR Patel', 'AS Joseph', 'AS Rajpoot', 'AS Raut', 'AS Roy', 'AS Yadav', 'AT Carey', 'AT Rayudu', 'AUK Pathan', 'Abdul Samad', 'Abhishek Sharma', 'Akash Deep', 'Aman Hakim Khan', 'Anand Rajan', 'Anirudh Singh', 'Ankit Sharma', 'Ankit Soni', 'Anmolpreet Singh', 'Anuj Rawat', 'Anureet Singh', 'Arshdeep Singh', 

In [4]:
players[:2]

['A Ashish Reddy', 'A Badoni']

In [5]:
import data_manipulation
import pandas as pd
player_name = players[0]
c = 1
first_match_date, last_match_date = data_manipulation.get_first_and_last_match_dates(player_name)
combined_df_batting = pd.DataFrame(data_manipulation.batter_stats(player_name, first_match_date, last_match_date))
combined_df_bowling = pd.DataFrame(data_manipulation.bowler_stats(player_name, first_match_date, last_match_date))
print("Epoch = ",c,"/",len(players),"----",player_name)
for player_name in players[1:]:
    c+=1
    print("Epoch = ",c,"/",len(players),"----",player_name)
    first_match_date, last_match_date = data_manipulation.get_first_and_last_match_dates(player_name)
    batter_stats = data_manipulation.batter_stats(player_name, first_match_date, last_match_date)
    combined_df_batting = pd.concat([combined_df_batting, pd.DataFrame(batter_stats)], ignore_index=True)
    bowler_stats = data_manipulation.bowler_stats(player_name, first_match_date, last_match_date)
    combined_df_bowling = pd.concat([combined_df_bowling, pd.DataFrame(bowler_stats)], ignore_index=True)

Epoch =  1 / 606 ---- A Ashish Reddy
Epoch =  2 / 606 ---- A Badoni
Epoch =  3 / 606 ---- A Chandila
Epoch =  4 / 606 ---- A Chopra
Epoch =  5 / 606 ---- A Choudhary
Epoch =  6 / 606 ---- A Dananjaya
Epoch =  7 / 606 ---- A Flintoff
Epoch =  8 / 606 ---- A Kumble
Epoch =  9 / 606 ---- A Manohar
Epoch =  10 / 606 ---- A Mishra
Epoch =  11 / 606 ---- A Mithun
Epoch =  12 / 606 ---- A Mukund
Epoch =  13 / 606 ---- A Nehra
Epoch =  14 / 606 ---- A Nortje
Epoch =  15 / 606 ---- A Singh
Epoch =  16 / 606 ---- A Symonds
Epoch =  17 / 606 ---- A Uniyal
Epoch =  18 / 606 ---- A Zampa
Epoch =  19 / 606 ---- AA Bilakhia
Epoch =  20 / 606 ---- AA Chavan
Epoch =  21 / 606 ---- AA Jhunjhunwala
Epoch =  22 / 606 ---- AA Noffke
Epoch =  23 / 606 ---- AB Agarkar
Epoch =  24 / 606 ---- AB Barath
Epoch =  25 / 606 ---- AB Dinda
Epoch =  26 / 606 ---- AB McDonald
Epoch =  27 / 606 ---- AB de Villiers
Epoch =  28 / 606 ---- AC Blizzard
Epoch =  29 / 606 ---- AC Gilchrist
Epoch =  30 / 606 ---- AC Thomas
Ep

In [6]:
combined_df_batting

,Name,Matches,Innings,Runs,Balls,Average,Strike Rate,Not Outs
0,A Ashish Reddy,31,23,280,193,18.67,145.08,8
1,A Badoni,13,11,161,130,20.12,123.85,3
2,A Chandila,12,2,4,7,NaN,57.14,2
3,A Chopra,7,6,53,71,8.83,74.65,0
4,A Choudhary,5,3,25,20,25.00,125.00,2
...,...,...,...,...,...,...,...,...
601,YV Takawale,16,10,192,178,24.00,107.87,2
602,Yash Dayal,9,1,0,1,0.00,0.00,0
603,Yashpal Singh,8,4,47,66,11.75,71.21,0
604,Yuvraj Singh,132,126,2754,2122,24.81,129.78,15


In [7]:
combined_df_batting = combined_df_batting.rename(columns={'Strike Rate': 'Strike_Rate', 'Not Outs': 'Not_Outs'})
combined_df_batting

,Name,Matches,Innings,Runs,Balls,Average,Strike_Rate,Not_Outs
0,A Ashish Reddy,31,23,280,193,18.67,145.08,8
1,A Badoni,13,11,161,130,20.12,123.85,3
2,A Chandila,12,2,4,7,NaN,57.14,2
3,A Chopra,7,6,53,71,8.83,74.65,0
4,A Choudhary,5,3,25,20,25.00,125.00,2
...,...,...,...,...,...,...,...,...
601,YV Takawale,16,10,192,178,24.00,107.87,2
602,Yash Dayal,9,1,0,1,0.00,0.00,0
603,Yashpal Singh,8,4,47,66,11.75,71.21,0
604,Yuvraj Singh,132,126,2754,2122,24.81,129.78,15


In [8]:
combined_df_bowling

,Name,Matches,Innings,Runs,Wickets,Economy,Balls,Strike Rate,Average
0,A Ashish Reddy,31,20,396,18,9.07,262,14.56,22.00
1,A Badoni,13,2,11,2,5.50,12,6.00,5.50
2,A Chandila,12,12,242,11,6.21,234,21.27,22.00
3,A Chopra,7,0,0,0,NaN,0,NaN,NaN
4,A Choudhary,5,5,144,5,8.55,101,20.20,28.80
...,...,...,...,...,...,...,...,...,...
601,YV Takawale,16,0,0,0,NaN,0,NaN,NaN
602,Yash Dayal,9,9,296,11,9.25,192,17.45,26.91
603,Yashpal Singh,8,0,0,0,NaN,0,NaN,NaN
604,Yuvraj Singh,132,73,1077,36,7.44,869,24.14,29.92


In [9]:
combined_df_bowling = combined_df_bowling.rename(columns={'Strike Rate': 'Strike_Rate'})
combined_df_bowling

,Name,Matches,Innings,Runs,Wickets,Economy,Balls,Strike_Rate,Average
0,A Ashish Reddy,31,20,396,18,9.07,262,14.56,22.00
1,A Badoni,13,2,11,2,5.50,12,6.00,5.50
2,A Chandila,12,12,242,11,6.21,234,21.27,22.00
3,A Chopra,7,0,0,0,NaN,0,NaN,NaN
4,A Choudhary,5,5,144,5,8.55,101,20.20,28.80
...,...,...,...,...,...,...,...,...,...
601,YV Takawale,16,0,0,0,NaN,0,NaN,NaN
602,Yash Dayal,9,9,296,11,9.25,192,17.45,26.91
603,Yashpal Singh,8,0,0,0,NaN,0,NaN,NaN
604,Yuvraj Singh,132,73,1077,36,7.44,869,24.14,29.92


In [10]:
import sqlite3


conn = sqlite3.connect('./ipl_database.db')

# Write the DataFrame to a SQL database
combined_df_batting.to_sql('ipl_batter_list', conn, index=False, if_exists='replace')
combined_df_bowling.to_sql('ipl_bowler_list', conn, index=False, if_exists='replace')

# Close the connection
conn.close()

In [12]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('./ipl_database.db')

# Read the combined_df_batting table from the database
combined_df_batting = pd.read_sql_query("SELECT * FROM ipl_batter_list WHERE Balls >= 60 ORDER BY Average DESC", conn)

# Close the connection
conn.close()

# Display the DataFrame
print(combined_df_batting)


              Name  Matches  Innings  Runs  Balls  Average  Strike_Rate   
0       MN van Wyk        5        5   167    132    55.67       126.52  \
1         KL Rahul      109      100  3895   2861    46.93       136.14   
2         AC Voges        9        7   181    143    45.25       126.57   
3          HM Amla       16       16   577    407    44.38       141.77   
4    Iqbal Abdulla       49       13    88     84    44.00       104.76   
..             ...      ...      ...   ...    ...      ...          ...   
314     SL Malinga      122       25    88     99     5.50        88.89   
315      YS Chahal      131       20    37     86     5.29        43.02   
316       R Sharma       60       20    66     75     4.71        88.00   
317       RP Singh       82       30    52     76     3.47        68.42   
318       S Nadeem       72       22    39     87     2.79        44.83   

     Not_Outs  
0           2  
1          17  
2           3  
3           3  
4          11  
.. 

In [51]:
import sqlite3
import pandas as pd

def innings_by_innings_list_match_list_details(id_list):
    Date = []
    Venue = []
    Season = []
    MatchNumber = []
    City = []
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            Date, Venue, Season, MatchNumber, City   
        FROM 
            ipl_match_list
        WHERE 
            ID = ?
        GROUP BY 
            ID;
        '''
        cursor.execute(query, (ID,))
        x = cursor.fetchall()[0]
        Date.append(x[0])
        Venue.append(x[1])
        Season.append(x[2])
        MatchNumber.append(x[3])
        City.append(x[4])
        
   

    conn.close()

    return Date, Venue, Season, MatchNumber, City

def innings_by_innings_list_batting(name, id_list):
    batter,	Runs_Scored, Balls_Faced, Four_Count, Six_Count, Out, Dismissal_Type, fielders_involved, Bowler = [], [], [], [], [], [], [], [], []

    batter_dict = {}
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            batter AS Name,
            SUM(batsman_run) AS Runs_Scored,
            COUNT(CASE WHEN extra_type IS NULL OR extra_type = 'byes' OR extra_type = 'legbyes' OR extra_type = 'noballs' THEN 1 ELSE NULL END) AS Balls_Faced,
            SUM(CASE WHEN batsman_run = 4 THEN 1 ELSE 0 END) AS Four_Count,
            SUM(CASE WHEN batsman_run = 6 THEN 1 ELSE 0 END) AS Six_Count,
            COUNT(CASE WHEN isWicketDelivery = 1 THEN 1 ELSE NULL END) AS Out,
            MAX(CASE WHEN isWicketDelivery = 1 THEN kind ELSE NULL END) AS Dismissal_Type,
            MAX(CASE WHEN isWicketDelivery = 1 THEN fielders_involved ELSE NULL END) AS fielders_involved,
            MAX(CASE WHEN isWicketDelivery = 1 AND kind <> 'run out' THEN bowler ELSE NULL END) AS Bowler
            
        FROM 
            ipl_ball_by_ball
        WHERE 
            batter = ? AND ID = ?
        GROUP BY 
            batter;
        '''
        cursor.execute(query, (name, ID))
        x = cursor.fetchall()[0]
        if x:
            batter.append(x[0])
            Runs_Scored.append(x[1])
            Balls_Faced.append(x[2])
            Four_Count.append(x[3])
            Six_Count.append(x[4])
            Out.append(x[5])
            Dismissal_Type.append(x[6])
            fielders_involved.append(x[7])
            Bowler.append(x[8])
        else:
            batter.append(None)
            Runs_Scored.append(None)
            Balls_Faced.append(None)
            Four_Count.append(None)
            Six_Count.append(None)
            Out.append(None)
            Dismissal_Type.append(None)
            fielders_involved.append(None)
            Bowler.append(None)
    
    Date, Venue, Season, MatchNumber, City = innings_by_innings_list_match_list_details(id_list)
    batter_dict["Date"] = Date
    batter_dict["MatchNumber"] = MatchNumber
    batter_dict["Season"] = Season
    batter_dict["batter"] = batter
    batter_dict["Runs_Scored"] = Runs_Scored
    batter_dict["Balls_Faced"] = Balls_Faced
    batter_dict["Four_Count"] = Four_Count
    batter_dict["Six_Count"] = Six_Count
    batter_dict["Out"] = Out
    batter_dict["Dismissal_Type"] = Dismissal_Type
    batter_dict["fielders_involved"] = fielders_involved
    batter_dict["Bowler"] = Bowler
    batter_dict["Venue"] = Venue
    batter_dict["City"] = City
    batter_dict["ID"] = id_list
    
    conn.close()
    df = pd.DataFrame(batter_dict)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date', ascending=True)
    return df

innings_by_innings_list_batting("Shubman Gill", [1254117, 1304103])

,Date,MatchNumber,Season,batter,Runs_Scored,Balls_Faced,Four_Count,Six_Count,Out,Dismissal_Type,fielders_involved,Bowler,Venue,City,ID
0,2021-10-15,Final,2021,Shubman Gill,51,43,6,0,1,lbw,None,DL Chahar,Dubai International Cricket Stadium,Dubai,1254117
1,2022-05-10,57,2022,Shubman Gill,63,49,7,0,0,None,None,None,Maharashtra Cricket Association Stadium,Pune,1304103


In [1]:
id_list = [1216536, 1216533, 1216509, 1216528, 1216525, 1216501, 1216513, 1216516, 1216539, 1216496, 1216492, 1181768, 1181767, 1181764, 1178430, 1178425, 1178419, 1178416, 1178414, 1178408, 1178404, 1178400, 1178398, 1178393, 1175370, 1175367, 1175360, 1175356, 1136620, 1136617, 1136612, 1136606, 1136603, 1136595, 1136593, 1136590, 1136587, 1136584, 1136580, 1136577, 1136572, 1136565, 1136561, 1082646, 1082633, 1082628, 1082610, 1082607, 1082598, 1082595, 1082591, 981019, 981013, 981011, 980999, 980995, 980987, 980981, 980977, 980969, 980959, 980953, 980937, 980931, 980927, 980921, 980907, 829819, 829811, 829797, 829787, 829777, 829769, 829763, 829749, 829741, 829735, 734041, 734033, 734017, 734003, 733999, 733989, 733979, 733975, 729315, 729305, 729297, 729291, 729285, 598072, 598071, 598065, 598063, 598058, 598052, 598049, 598047, 598043, 598036, 598032, 598026, 598023, 598019, 598014, 598005, 548377, 548373, 548365, 548361, 548358, 548354, 548349, 501263, 501258, 501252, 501249, 501240, 501235, 501231, 501225, 501220, 501214, 501209, 419158, 419154, 419150, 419143, 419141, 419137, 336040, 336038, 336019, 336036, 336027, 336023, 336015, 336011, 336008, 336005, 336000, 335992, 335990, 335987, 335984]

In [2]:
team1_list = ['Royal Challengers Bangalore', 'Rajasthan Royals', 'Chennai Super Kings']
team2_list = ['Sunrisers Hyderabad', 'Rising Pune Supergiant', 'Rajasthan Royals', 'Delhi Daredevils', 'Rising Pune Supergiants', 'Delhi Capitals', 'Gujarat Lions', 'Kings XI Punjab', 'Kochi Tuskers Kerala', 'Deccan Chargers', 'Chennai Super Kings', 'Kolkata Knight Riders', 'Mumbai Indians', 'Pune Warriors', 'Royal Challengers Bangalore']

In [3]:
name = "SR Watson"

In [4]:
venue = ["M.Chinnaswamy Stadium", "Sawai Mansingh Stadium", "MA Chidambaram Stadium"]

In [13]:
import sqlite3
import pandas as pd
def select_matches_home(player_name, id_list, teamfor_list, teamagainst_list, venue):

    # Connect to the database
    conn = sqlite3.connect('./ipl_database.db')
    
    # Format teamfor_list_str for the query
    teamfor_list_str = ', '.join([f'"{team}"' for team in teamfor_list])

    # Format teamagainst_list_str for the query
    teamagainst_list_str = ', '.join([f'"{team}"' for team in teamagainst_list])

    # Format id_list_str for the query
    id_list_str = ', '.join([str(id) for id in id_list])
    print(type(teamfor_list_str))
    print(type(teamagainst_list_str))
    print(type(id_list_str))   

    # SQL query
    query = f'''
    SELECT m.ID, 
    CASE 
        WHEN p1 LIKE '%' || ? || '%' THEN Team1 
        WHEN p2 LIKE '%' || ? || '%' THEN Team2 
    END AS PlayerTeam,
    CASE 
        WHEN p1 LIKE '%' || ? || '%' THEN Team2 
        WHEN p2 LIKE '%' || ? || '%' THEN Team1 
    END AS OpponentTeam,
    v.City, 
    m.Date
    FROM ipl_match_list m
    JOIN ipl_venue_list v ON m.Venue = v.Venue
    CROSS JOIN (SELECT ? AS p1, ? AS p2) AS params
    WHERE (Team1Players LIKE '%' || ? || '%' OR Team2Players LIKE '%' || ? || '%')
    AND v.Teams LIKE '%' || ? || '%';
    '''

    # Fetch results into DataFrame
    df = pd.read_sql_query(query, conn, params=('%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', '%' + player_name + '%', teamfor_list_str, teamagainst_list_str, id_list_str, '%' + venue + '%'))

    # Close the connection
    conn.close()

    return df

In [14]:
select_matches_home(name, id_list, team1_list, team2_list, venue)

<class 'str'>
<class 'str'>
<class 'str'>


TypeError: can only concatenate str (not "list") to str

In [3]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ipl_database.db')

# Read ipl_match_list and ipl_ball_by_ball tables into pandas dataframes
ipl_match_list = pd.read_sql_query("SELECT * FROM ipl_match_list", conn)
ipl_ball_by_ball = pd.read_sql_query("SELECT * FROM ipl_ball_by_ball", conn)

# Close the database connection
conn.close()

# Now you can use ipl_match_list and ipl_ball_by_ball dataframes for analysis

# Assuming you have loaded the ipl_match_list and ipl_ball_by_ball dataframes

# Step 1: Merge the dataframes
merged_data = pd.merge(ipl_match_list, ipl_ball_by_ball, on='ID')

# Step 2: Filter based on venue
venue_name = 'Wankhede Stadium'  # Replace 'Your Venue Name' with the actual venue name
venue_data = merged_data[merged_data['Venue'] == venue_name]


# Step 3: Group by batsman and aggregate batting statistics
batting_stats = venue_data.groupby('batter').agg(
    runs=('batsman_run', 'sum'),
    dismissals=('isWicketDelivery', 'sum'),  # Count dismissals
    balls=('ballnumber', 'count'),
    fours=('batsman_run', lambda x: (x == 4).sum()),
    sixes=('batsman_run', lambda x: (x == 6).sum()),
    strike_rate=('batsman_run', lambda x: (x.sum() / len(x)) * 100)
).reset_index()

batting_stats['Average'] = batting_stats.apply(lambda x: x['runs'] / x['dismissals'] if x['dismissals'] != 0 else float('nan'), axis=1)


batting_stats = batting_stats.sort_values(by='runs', ascending=False)
batting_stats

,batter,runs,dismissals,balls,fours,sixes,strike_rate,Average
253,RG Sharma,1840,55,1424,167,73,129.213483,33.454545
136,KA Pollard,1228,44,831,78,85,147.773767,27.909091
28,AT Rayudu,988,41,786,88,43,125.699746,24.097561
139,KD Karthik,772,25,577,83,25,133.795494,30.880000
119,JC Buttler,728,19,507,70,40,143.589744,38.315789
...,...,...,...,...,...,...,...,...
157,LR Shukla,0,1,4,0,0,0.000000,0.000000
168,M Prasidh Krishna,0,0,1,0,0,0.000000,NaN
171,M Theekshana,0,1,3,0,0,0.000000,0.000000
48,C Sakariya,0,1,1,0,0,0.000000,0.000000


In [4]:
jc_buttler_stats = batting_stats[batting_stats['batter'] == "JC Buttler"]
jc_buttler_stats

,batter,runs,dismissals,balls,fours,sixes,strike_rate,Average
119,JC Buttler,728,19,507,70,40,143.589744,38.315789


In [5]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ipl_database.db')

# Read ipl_match_list and ipl_ball_by_ball tables into pandas dataframes
ipl_match_list = pd.read_sql_query("SELECT * FROM ipl_match_list", conn)
ipl_ball_by_ball = pd.read_sql_query("SELECT * FROM ipl_ball_by_ball", conn)
ipl_match_list['Venue'] = ipl_match_list['Venue'].replace('M.Chinnaswamy Stadium', 'M Chinnaswamy Stadium')
# Close the database connection
conn.close()

# Merge the dataframes
merged_data = pd.merge(ipl_match_list, ipl_ball_by_ball, on='ID')

# Filter data for matches where "SR Tendulkar" played
player_name = "SR Tendulkar"
player_data = merged_data[merged_data['batter'] == player_name]

# Group by venue and aggregate batting statistics
venue_stats = player_data.groupby('Venue').agg(
    runs=('batsman_run', 'sum'),
    dismissals=('isWicketDelivery', 'sum'),
    balls=('ballnumber', 'count'),
    fours=('batsman_run', lambda x: (x == 4).sum()),
    sixes=('batsman_run', lambda x: (x == 6).sum()),
    strike_rate=('batsman_run', lambda x: (x.sum() / len(x)) * 100)
)

# Calculate average runs per dismissal
venue_stats['Average'] = venue_stats['runs'] / venue_stats['dismissals'].replace(0, pd.NA)

# Reset index to make Venue a column instead of index
venue_stats.reset_index(inplace=True)

# Display the venue-wise batting statistics
print(venue_stats)

                                 Venue  runs  dismissals  balls  fours  sixes   
0                    Brabourne Stadium   261           6    211     35      1  \
1                         Buffalo Park    49           2     47      5      2   
2           Dr DY Patil Sports Academy   136           4    129     23      0   
3                         Eden Gardens    42           3     36      6      0   
4                     Feroz Shah Kotla   188           5    162     21      4   
5                            Kingsmead    77           3     66      7      3   
6                M Chinnaswamy Stadium   148           4    144     20      1   
7               MA Chidambaram Stadium   101           4     77     14      1   
8                New Wanderers Stadium    11           1     11      2      0   
9                             Newlands    59           0     49      7      0   
10  Punjab Cricket Association Stadium    69           3     63      8      0   
11  Rajiv Gandhi Internation

In [6]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ipl_database.db')

# Read ipl_match_list and ipl_ball_by_ball tables into pandas dataframes
ipl_match_list = pd.read_sql_query("SELECT * FROM ipl_match_list", conn)
ipl_ball_by_ball = pd.read_sql_query("SELECT * FROM ipl_ball_by_ball", conn)
ipl_match_list['Venue'] = ipl_match_list['Venue'].replace('M.Chinnaswamy Stadium', 'M Chinnaswamy Stadium')
# Close the database connection
conn.close()

# Merge the dataframes
merged_data = pd.merge(ipl_match_list, ipl_ball_by_ball, on='ID')

# Filter data for matches where "SL Malinga" bowled
player_name = "SL Malinga"
player_data = merged_data[merged_data['bowler'] == player_name]

# Group by venue and aggregate bowling statistics
venue_stats = player_data.groupby('Venue').agg(
    runs_conceded=('total_run', 'sum'),
    wickets_taken=('isWicketDelivery', 'sum'),
    balls=('ballnumber', 'count'),
    fours=('total_run', lambda x: ((x == 4) | (x == 5)).sum()),  # Assuming 5 denotes no-ball in total_run
    sixes=('total_run', lambda x: ((x == 6) | (x == 7)).sum()),  # Assuming 7 denotes wide in total_run
    strike_rate=('total_run', lambda x: (x.sum() / len(x)) * 100)
)

# Calculate average runs per wicket
venue_stats['Average'] = venue_stats['runs_conceded'] / venue_stats['wickets_taken'].replace(0, pd.NA)

# Reset index to make Venue a column instead of index
venue_stats.reset_index(inplace=True)

# Display the venue-wise bowling statistics
print(venue_stats)

                                                Venue  runs_conceded   
0                                Arun Jaitley Stadium             41  \
1                                    Barabati Stadium             30   
2                                   Brabourne Stadium            171   
3                                        Buffalo Park             53   
4                          Dr DY Patil Sports Academy             96   
5   Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...             27   
6                 Dubai International Cricket Stadium             81   
7                                        Eden Gardens            247   
8                                    Feroz Shah Kotla            168   
9        Himachal Pradesh Cricket Association Stadium             40   
10                             Holkar Cricket Stadium             58   
11                                          Kingsmead             68   
12                              M Chinnaswamy Stadium           

In [12]:
import pandas as pd
import numpy as np
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ipl_database.db')

# Read ipl_match_list and ipl_ball_by_ball tables into pandas dataframes
ipl_match_list = pd.read_sql_query("SELECT * FROM ipl_match_list", conn)
ipl_ball_by_ball = pd.read_sql_query("SELECT * FROM ipl_ball_by_ball", conn)
ipl_match_list['Venue'] = ipl_match_list['Venue'].replace('M.Chinnaswamy Stadium', 'M Chinnaswamy Stadium')
# Close the database connection
conn.close()

# Merge the dataframes
merged_data = pd.merge(ipl_match_list, ipl_ball_by_ball, on='ID')

# Check for NA or NaN values in the 'fielders_involved' column
merged_data['fielders_involved'] = merged_data['fielders_involved'].fillna('')  # Fill NA or NaN with empty string

# Filter data for matches where the player was involved in fielding (caught, caught & bowled, run out, stumped)
player_name = "MS Dhoni"  # Specify the player's name
fielding_data = merged_data[(merged_data['fielders_involved'].str.contains(player_name)) | 
                            (merged_data['kind'].isin(['caught', 'caught and bowled', 'run out', 'stumped']))]

# Group by venue and aggregate fielding statistics
venue_stats = fielding_data.groupby('Venue').agg(
    catches=('kind', lambda x: ((x == 'caught') | (x == 'caught and bowled')).sum()),
    run_outs=('kind', lambda x: (x == 'run out').sum()),
    stumpings=('kind', lambda x: (x == 'stumped').sum())
)

# Reset index to make Venue a column instead of index
venue_stats.reset_index(inplace=True)

# Count the number of matches played by the player in each venue
matches_played = merged_data[merged_data['fielders_involved'].str.contains(player_name, na=False)].groupby('Venue')['ID'].nunique().reset_index()
matches_played.columns = ['Venue', 'Matches_Played']

# Merge matches played with venue_stats
venue_stats = pd.merge(venue_stats, matches_played, on='Venue', how='left')  # Use 'left' join to keep all venues in venue_stats

# Display the result
print(venue_stats)

                                                Venue  catches  run_outs   
0                                Arun Jaitley Stadium      136        15  \
1                                    Barabati Stadium       43        10   
2                                   Brabourne Stadium      214        25   
3                                        Buffalo Park       23         4   
4                               De Beers Diamond Oval       17         7   
5                          Dr DY Patil Sports Academy      346        30   
6   Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...      104         8   
7                 Dubai International Cricket Stadium      379        39   
8                                        Eden Gardens      533        93   
9                                    Feroz Shah Kotla      422        78   
10                                         Green Park       27         6   
11       Himachal Pradesh Cricket Association Stadium       73         2   
12          

In [2]:
import pandas as pd

# Sample DataFrame
data = {'Matches': [222], 'Innings': [215], 'Runs': [6634], 'Balls': [5133], 'Average': [36.25], 'Strike Rate': [129.24], 'Not Outs': [32]}
df = pd.DataFrame(data)

# Label the index as "Serial No."
df_bat = df.rename_axis('Serial No.')

df_bat

,Matches,Innings,Runs,Balls,Average,Strike Rate,Not Outs
Serial No.,,,,,,,
0,222,215,6634,5133,36.25,129.24,32


In [13]:
def player_bowling_stats_season_wise(player_name, start_date, end_date):
    # Connect to the SQLite database
    conn = sqlite3.connect('ipl_database.db')

    # Read ipl_match_list and ipl_ball_by_ball tables into pandas dataframes
    ipl_match_list = pd.read_sql_query("SELECT * FROM ipl_match_list", conn)
    ipl_ball_by_ball = pd.read_sql_query("SELECT * FROM ipl_ball_by_ball", conn)
    
    # Filter matches based on start_date and end_date
    ipl_match_list['Date'] = pd.to_datetime(ipl_match_list['Date'])
    ipl_match_list = ipl_match_list[(ipl_match_list['Date'] >= start_date) & (ipl_match_list['Date'] <= end_date)]

    # Close the database connection
    conn.close()

    # Merge the dataframes
    merged_data = pd.merge(ipl_match_list, ipl_ball_by_ball, on='ID')

    # Filter data for matches where the specified player bowled
    player_data = merged_data[merged_data['bowler'] == player_name]

    # Group by venue and aggregate bowling statistics
    season_stats = player_data.groupby('Season').agg(
        runs_conceded=('total_run', 'sum'),
        wickets_taken=('isWicketDelivery', 'sum'),
        balls_bowled=('ballnumber', 'count'),
        fours=('total_run', lambda x: ((x == 4) | (x == 5)).sum()),  # Assuming 5 denotes no-ball in total_run
        sixes=('total_run', lambda x: ((x == 6) | (x == 7)).sum()),  # Assuming 7 denotes wide in total_run
    )

    # Calculate bowling strike rate
    season_stats['Strike_Rate'] = season_stats['balls_bowled'] / season_stats['wickets_taken']

    # Calculate average runs per wicket
    season_stats['Average'] = season_stats['runs_conceded'] / season_stats['wickets_taken'].replace(0, pd.NA)

    # Reset index to make Venue a column instead of index
    season_stats.reset_index(inplace=True)

    # Display the venue-wise bowling statistics
    return season_stats

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('./ipl_database.db')
cursor = conn.cursor()

# Update the venue names
update_query = """
UPDATE ipl_match_list
SET Venue = 'M Chinnaswamy Stadium'
WHERE Venue = 'M.Chinnaswamy Stadium';
"""

# Execute the update query
cursor.execute(update_query)

# Commit the changes
conn.commit()

# Close the connection
conn.close()

print("Venue names updated successfully.")